### Data Prepration

In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
import cv2

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import  tensorflow.keras.layers as layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix, classification_report


from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D
)

### Data load

In [ ]:
import pickle
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
            plt.imshow(X[0], cmap='gray')  
            plt.show()  
            print(y[0])


### Define the Model

In [ ]:
input_shape = (300, 300, 3)
n_classes = 4

model = Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [ ]:
model.summary()

### Compile the model

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

###  Train the Model


In [ ]:
es_callback = EarlyStopping(
    monitor = 'val_loss',
    patience=5,
    restore_best_weights = True
    
)

In [ ]:
tb_callback = TensorBoard(log_dir = "../tensorboard_logs", histogram_freq = 1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)


In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), validation_steps=6, verbose=1, 
                    epochs=50,callbacks=[es_callback,tb_callback])


In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'], label="train_accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.plot(history.history['loss'], label="train_loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
predictions = model.predict(x_test)

In [ ]:
df1=pd.DataFrame(predictions)

In [ ]:
pred=df1.idxmax(axis=1)


In [ ]:
pred

### Define the model using keras hypertuning 

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(layers.InputLayer(input_shape=input_shape)),
  model.add(layers.Conv2D(32, kernel_size = (3,3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Conv2D(64,  kernel_size = (3,3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Conv2D(64,  kernel_size = (3,3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Conv2D(64, (3, 3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Conv2D(64, (3, 3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Conv2D(64, (3, 3), activation='relu')),
  model.add(layers.MaxPooling2D((2, 2))),
  model.add(layers.Flatten()),

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(x_train, y_train)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
plt.plot(history.history['accuracy'], label="train_accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.plot(history.history['loss'], label="train_loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
cm = confusion_matrix(y_test, pred)

In [ ]:
plt.figure(dpi=70)
sns.heatmap(cm, annot=True, cmap="Blues")
plt.xticks(ticks=[0.5,1.5,2.5,3.5], labels=["Chickenpox", "Measles", "Monkeypox","Normal"])
plt.yticks(ticks=[0.5,1.5,2.5,3.5], labels=["Chickenpox", "Measles", "Monkeypox","Normal"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

